# 使用Amazon SageMaker Processing Job进行特征转换

通常，一个机器学习（ML）过程包括几个步骤。首先，通过各种ETL任务收集数据，然后对数据进行预处理，通过采用标准技术或先前的知识对数据集进行特征化，最后使用算法训练一个ML模型。

我们可以使用分布式数据处理框架，如Scikit-Learn、Spark、Ray等，对数据集进行预处理，以便为训练做好准备。在这个笔记本中，我们将使用Amazon SageMaker Processing，并利用HuggingFace在托管的SageMaker环境中运行我们的处理工作负载。

<a name='1'></a>
## Set up Kernel and Required Dependencies

First, check that the correct kernel is chosen.

<img src="img/kernel_set_up.png" width="300"/>

You can click on that to see and check the details of the image, kernel, and instance type.

<img src="img/w3_kernel_and_instance_type.png" width="600"/>

# NOTE:  THIS NOTEBOOK WILL TAKE A 5-10 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

## Contents

1. 设置环境
2. 设置输入数据
3. 设置输出数据
4. 为运行处理作业构建一个Scikit-Learn容器
5. 使用Amazon SageMaker运行Processing job
6. 检查处理后的输出数据

# 设置环境

开始以下内容：

* 用于训练和模型数据的S3存储桶和前缀。使用Amazon SageMaker会话指定的默认存储桶。
* 用于给处理和训练提供数据集访问权限的IAM角色ARN。

In [2]:
# 导入 Amazon SageMaker 和 boto3 库
import sagemaker
import boto3

# 创建一个 SageMaker 会话
sess = sagemaker.Session()

# 获取当前 SageMaker 执行角色的 Amazon Resource Name（ARN），该角色决定了 SageMaker 可以对哪些 AWS 资源进行访问
role = sagemaker.get_execution_role()

# 获取当前 SageMaker 会话的默认 S3 存储桶名
bucket = sess.default_bucket()

# 获取当前 boto3 会话的区域名，这决定了你的 AWS 服务在哪个地理位置被部署
region = boto3.Session().region_name

# 导入 botocore 库的 config 模块，它提供了设置 AWS 服务客户端的配置选项
import botocore.config

# 创建一个 botocore 的 Config 对象，设置 user_agent_extra 参数为 'dsoaws/2.0'，这将在每次 AWS 请求的 User-Agent 头部添加这个字符串
config = botocore.config.Config(
    user_agent_extra='dsoaws/2.0'
)

# 创建一个指向 Amazon SageMaker 的 boto3 客户端，设置服务名为 "sagemaker"，区域名为前面获取的 region，配置为前面创建的 config
sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region, 
                            config=config)

# 创建一个指向 Amazon S3 的 boto3 客户端，设置服务名为 "s3"，区域名为前面获取的 region，配置为前面创建的 config
s3 = boto3.Session().client(service_name="s3", 
                            region_name=region,
                            config=config)

# 设置输入数据的S3 URI

In [3]:
%store -r raw_input_data_s3_uri

In [4]:
!aws s3 ls $raw_input_data_s3_uri

2023-09-05 03:06:01    6544107 dialogsum-1.csv
2023-09-05 03:06:01    6572423 dialogsum-2.csv


In [5]:
%store raw_input_data_s3_uri

Stored 'raw_input_data_s3_uri' (str)


# 加载其他变量

In [6]:
%store -r model_checkpoint

In [7]:
try:
    model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
print(model_checkpoint)

google/flan-t5-base


# 使用Amazon SageMaker运行处理作业Processing Job

接下来，使用Amazon SageMaker Python SDK提交一个使用我们自定义python脚本的处理作业。

# 查看处理脚本

In [10]:
# !pygmentize 是一个命令行工具，用于将源代码高亮显示。
# 它是 pygments 包的一部分，pygments 是一个 Python 语法高亮的库。该命令需要在命令行环境中运行，例如在一个终端或者 Jupyter notebook 的代码单元格中。
# 使用 !pygmentize preprocess.py 命令，将会将 preprocess.py 这个 Python 文件的内容以语法高亮的形式输出。
!pygmentize preprocess.py

# 导入 subprocess 模块，该模块可以用来创建新的进程，并连接到其输入/输出/错误管道，获取返回值等
import subprocess

# 导入 sys 模块，该模块提供对 Python 解释器使用或维护的一些变量的访问
import sys

# 导入 json 模块，该模块提供了 JSON 数据解析的方法
import json

# 导入 argparse 模块，该模块提供了创建命令行参数和选项的方法
import argparse

# 使用 subprocess 模块的 check_call 方法运行 pip 安装命令，安装特定版本的 transformers、datasets 和 torch 库
# sys.executable 是 Python 解释器的路径，"-m" 是一个命令行选项，表示后面的字符串应作为模块名来执行
subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers==4.26.1", "datasets==2.9.0", "torch==1.13.1"])

# 从 transformers 库中导入 AutoTokenizer 类，该类可以自动加载预训练的分词器
from transformers import AutoTokenizer

# 从 datasets 库中导入 load_dataset 函数和 DatasetDict 类，load_dataset 函数可以加载各种 NLP 数据集，DatasetDict 类可以管理多个数据集
from datasets import load_dataset, DatasetDict

# 导入 os 模块，该模块提供了大量的函数来处理文件和目录
import os

# 导入 time 模块，该模块提供了时间相关的函数
import time


# 定义一个 transform_dataset 函数，参数包括输入数据的路径、输出数据的路径、预训练模型名，以及训练集、测试集和验证集的划分比率
def transform_dataset(input_data,
                      output_data,
                      hug

将此脚本作为Processing job运行。你还需要指定一个`ProcessingInput`，其`source`参数为Amazon S3存储桶，`destination`是脚本从`/opt/ml/processing/input`（在Docker容器内）读取此数据的位置。处理容器内的所有本地路径必须以`/opt/ml/processing/`开头。

还需要给`run()`方法一个`ProcessingOutput`，其中`source`是脚本写入输出数据的路径。对于输出，`destination`默认为Amazon SageMaker Python SDK为你创建的一个S3存储桶，格式为`s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`。你也可以给`ProcessingOutput`提供`output_name`值，这样在作业运行后更容易检索这些输出工件。

`run()`方法中的参数是`preprocess.py`脚本中的命令行参数。

注意，我们使用`ShardedByS3Key`对数据进行分片，以在集群中的所有工作节点上分布转换操作。

In [11]:
# 设置 SageMaker Processing Job 的一些参数
# 处理任务使用的机器类型为 "ml.c5.2xlarge"
processing_instance_type = "ml.c5.2xlarge"

# 在处理任务中使用的机器数量为 2，即并行执行处理任务的实例数
processing_instance_count = 2

# 训练集划分的比例为 90%
train_split_percentage = 0.9

# 验证集划分的比例为 5%
validation_split_percentage = 0.05

# 测试集划分的比例为 5%
test_split_percentage = 0.05

In [12]:
# 创建一个 SKLearnProcessor 实例，这是一个用于运行 SageMaker Processing Jobs 的处理器。处理器会在指定的实例上运行处理任务。

# 导入 SKLearnProcessor 类
from sagemaker.sklearn.processing import SKLearnProcessor

# 创建一个 SKLearnProcessor 实例
processor = SKLearnProcessor(
    # 指定 Scikit-learn 框架的版本
    framework_version="0.23-1",
    
    # 指定用于执行 SageMaker Processing Job 的 AWS IAM 角色
    role=role,
    
    # 指定用于执行处理任务的实例类型
    instance_type=processing_instance_type,
    
    # 指定用于执行处理任务的实例数量
    instance_count=processing_instance_count,
    
    # 指定环境变量，这些环境变量将被传递到处理容器中
    env={"AWS_DEFAULT_REGION": region},
    
    # 指定处理任务的最大运行时间，单位为秒
    max_runtime_in_seconds=7200,
)

In [13]:
# 生成一个 S3 路径的字符串，这个路径指向名为 'data-summarization' 的目录，
# 这个目录位于名为 'bucket' 的 S3 存储桶中
input_s3 = f's3://{bucket}/data-summarization/'

In [14]:
!aws s3 ls {input_s3}

2023-09-05 03:06:01    6544107 dialogsum-1.csv
2023-09-05 03:06:01    6572423 dialogsum-2.csv


In [15]:
# 导入 ProcessingInput 和 ProcessingOutput 类
from sagemaker.processing import ProcessingInput, ProcessingOutput

# 使用 processor（一个 SKLearnProcessor 实例）运行处理任务
processor.run(
    # 指定用于执行处理任务的 Python 脚本
    code="preprocess.py",
    # 指定处理任务的输入数据
    inputs=[
        ProcessingInput(
            # 输入数据的名字，可以在处理脚本中使用这个名字来引用这个输入数据
            input_name="raw-input-data",
            
            # 输入数据的 S3 URI
            source=raw_input_data_s3_uri,
            
            # 输入数据将被下载到处理容器的这个目录中
            destination="/opt/ml/processing/input/data/",
            
            # 输入数据的 S3 分发类型，"ShardedByS3Key" 表示数据将被均匀地分发到各个实例
            s3_data_distribution_type="ShardedByS3Key",
        )
    ],
    # 指定处理任务的输出数据
    outputs=[
        ProcessingOutput(
            # 输出数据的名字，可以在处理脚本中使用这个名字来引用这个输出数据
            output_name="train", 
            
            # 输出数据的 S3 上传模式，"EndOfJob" 表示数据将在处理任务结束后上传到 S3
            s3_upload_mode="EndOfJob", 
            
            # 处理任务的输出数据将被写入到这个目录中，然后被上传到 S3
            source="/opt/ml/processing/output/data/train"
        ),
        ProcessingOutput(
            output_name="validation",
            s3_upload_mode="EndOfJob",
            source="/opt/ml/processing/output/data/validation",
        ),
        ProcessingOutput(
            output_name="test", 
            s3_upload_mode="EndOfJob", 
            source="/opt/ml/processing/output/data/test"
        ),
    ],
    # 传递给处理脚本的命令行参数
    arguments=[
        "--train-split-percentage",
        str(train_split_percentage),
        "--validation-split-percentage",
        str(validation_split_percentage),
        "--test-split-percentage",
        str(test_split_percentage),
        "--model-checkpoint",
        str(model_checkpoint),
    ],
    # 是否在控制台打印处理任务的日志
    logs=True,
    
    # 是否等待处理任务完成
    wait=False,
)

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2023-09-08-16-00-28-289


In [16]:
# 获取 processor运行的所有处理任务（jobs）列表中的最后一个任务，
# 然后调用 describe() 方法获取这个处理任务的详细信息，
# 最后从详细信息中获取处理任务的名称（"ProcessingJobName"）
scikit_processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]

# 打印处理任务的名称
print(scikit_processing_job_name)

sagemaker-scikit-learn-2023-09-08-16-00-28-289


In [17]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, scikit_processing_job_name, region
        )
    )
)

# 监控Processing Job

In [18]:
# 使用 ProcessingJob 类的 from_processing_name 方法根据处理任务的名称创建一个 ProcessingJob 实例。
# 这个方法需要两个参数：处理任务的名称和一个 SageMaker 会话（sagemaker_session）。
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=scikit_processing_job_name, sagemaker_session=sess
)

# 调用 ProcessingJob 实例的 describe 方法获取处理任务的详细信息。
# 这个方法返回一个包含了处理任务详细信息的字典。
processing_job_description = running_processor.describe()

# 打印处理任务的详细信息。
print(processing_job_description)

{'ProcessingInputs': [{'InputName': 'raw-input-data', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-941797585610/data-summarization/', 'LocalPath': '/opt/ml/processing/input/data/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'train', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/train', 'LocalPath': '/opt/ml/processing/output/data/train', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}, {'OutputName': 'validation', 'S3O

In [19]:
# 使用 ProcessingJob 实例的 wait 方法等待处理任务完成。
# 这个方法会阻塞当前线程，直到处理任务完成。
# 参数 logs=False 表示在等待处理任务完成时不打印处理任务的日志。
running_processor.wait(logs=False)

!

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

# 检查处理后的输出数据

查看转换后数据集，以确保处理成功。

In [20]:
# 再次调用 ProcessingJob 实例的 describe 方法获取处理任务的详细信息。
# 这个方法返回一个包含了处理任务详细信息的字典。
processing_job_description = running_processor.describe()

# 从处理任务的详细信息中获取输出数据的配置信息。
output_config = processing_job_description["ProcessingOutputConfig"]

# 遍历输出数据的配置信息
for output in output_config["Outputs"]:
    # 如果输出数据的名字是 "train"，那么获取这个输出数据的 S3 URI，并保存到变量 processed_train_data_s3_uri 中
    if output["OutputName"] == "train":
        processed_train_data_s3_uri = output["S3Output"]["S3Uri"]
    # 如果输出数据的名字是 "validation"，那么获取这个输出数据的 S3 URI，并保存到变量 processed_validation_data_s3_uri 中
    if output["OutputName"] == "validation":
        processed_validation_data_s3_uri = output["S3Output"]["S3Uri"]
    # 如果输出数据的名字是 "test"，那么获取这个输出数据的 S3 URI，并保存到变量 processed_test_data_s3_uri 中
    if output["OutputName"] == "test":
        processed_test_data_s3_uri = output["S3Output"]["S3Uri"]

# 打印处理后的训练数据的 S3 URI
print(processed_train_data_s3_uri)
# 打印处理后的验证数据的 S3 URI
print(processed_validation_data_s3_uri)
# 打印处理后的测试数据的 S3 URI
print(processed_test_data_s3_uri)

s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/train
s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/validation
s3://sagemaker-us-east-1-941797585610/sagemaker-scikit-learn-2023-09-08-16-00-28-289/output/test


In [21]:
!aws s3 ls $processed_train_data_s3_uri/

2023-09-08 16:07:05    2540571 1694189218128.parquet
2023-09-08 16:07:04    2545157 1694189219320.parquet


In [22]:
!aws s3 ls $processed_validation_data_s3_uri/

2023-09-08 16:07:05     150701 1694189218128.parquet
2023-09-08 16:07:04     150220 1694189219320.parquet


In [23]:
!aws s3 ls $processed_test_data_s3_uri/

2023-09-08 16:07:06     157115 1694189218128.parquet
2023-09-08 16:07:05     153865 1694189219320.parquet


# 将变量传递给下一个笔记本

In [24]:
%store raw_input_data_s3_uri

Stored 'raw_input_data_s3_uri' (str)


In [25]:
%store train_split_percentage

Stored 'train_split_percentage' (float)


In [26]:
%store validation_split_percentage

Stored 'validation_split_percentage' (float)


In [27]:
%store test_split_percentage

Stored 'test_split_percentage' (float)


In [ ]:
# %store balance_dataset

In [28]:
%store processed_train_data_s3_uri

Stored 'processed_train_data_s3_uri' (str)


In [29]:
%store processed_validation_data_s3_uri

Stored 'processed_validation_data_s3_uri' (str)


In [30]:
%store processed_test_data_s3_uri

Stored 'processed_test_data_s3_uri' (str)


In [31]:
%store

Stored variables and their in-db values:
ingest_create_athena_table_parquet_passed             -> True
local_data_processed_path                             -> './data-summarization-processed/'
model_checkpoint                                      -> 'google/flan-t5-base'
processed_test_data_s3_uri                            -> 's3://sagemaker-us-east-1-941797585610/sagemaker-s
processed_train_data_s3_uri                           -> 's3://sagemaker-us-east-1-941797585610/sagemaker-s
processed_validation_data_s3_uri                      -> 's3://sagemaker-us-east-1-941797585610/sagemaker-s
raw_input_data_s3_uri                                 -> 's3://sagemaker-us-east-1-941797585610/data-summar
role                                                  -> 'arn:aws:iam::941797585610:role/service-role/Amazo
s3_private_path_tsv                                   -> 's3://sagemaker-us-east-1-941797585610/amazon-revi
s3_public_path_tsv                                    -> 's3://dsoaws/tsv'
setu

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>